In [17]:
import numpy as np

In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2') # loading gpt2 from transformers library
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # loading gpt2 tokenizer from transformers library

In [14]:
input_text = "Once upon a time,"
input_ids = gpt2_tokenizer.encode(input_text, return_tensors='pt') # tokenize input
output = gpt2.generate(input_ids, max_length=50) # run inference
generated_text = gpt2_tokenizer.decode(output[0], skip_special_tokens=True) # decode output tokens
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world


In [16]:
state_dict = gpt2.state_dict()
for name, param in state_dict.items():
    print(name)
    print(type(param))
    print(param.shape)

transformer.wte.weight
<class 'torch.Tensor'>
torch.Size([50257, 768])
transformer.wpe.weight
<class 'torch.Tensor'>
torch.Size([1024, 768])
transformer.h.0.ln_1.weight
<class 'torch.Tensor'>
torch.Size([768])
transformer.h.0.ln_1.bias
<class 'torch.Tensor'>
torch.Size([768])
transformer.h.0.attn.c_attn.weight
<class 'torch.Tensor'>
torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias
<class 'torch.Tensor'>
torch.Size([2304])
transformer.h.0.attn.c_proj.weight
<class 'torch.Tensor'>
torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias
<class 'torch.Tensor'>
torch.Size([768])
transformer.h.0.ln_2.weight
<class 'torch.Tensor'>
torch.Size([768])
transformer.h.0.ln_2.bias
<class 'torch.Tensor'>
torch.Size([768])
transformer.h.0.mlp.c_fc.weight
<class 'torch.Tensor'>
torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias
<class 'torch.Tensor'>
torch.Size([3072])
transformer.h.0.mlp.c_proj.weight
<class 'torch.Tensor'>
torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias
<class 'tor